In [ ]:
# Pull data

# 
# All arguments in a thread

# get all sentences

# Either (Can also prune the comment trees before any of these):
# Pair all sentences n^2
# Pair some sentences (randomly)
# Pair sentences only between comment replies

# Predict the relations between all.

# Connect into graph



In [ ]:
# Retrieve data from r/changemyview using Reddit's API through the PRAW library
# Randomly select pairs of sentences from neighbouring 
# Saves the data to a file for labelling

import json
import re
import praw
import nltk
import random

# download dataset for sentence tokenizer
nltk.download('punkt')

#################### FETCH ####################

# Create new praw instance with credentials from praw.ini
reddit = praw.Reddit('arg-mining')

# Threads and their arguments
# key = submission_id
# value = list of arguments
arg_threads = {}

arg_titles = {}

# Add sentences from submission title, body and comments to arg_threads array
for submission in reddit.subreddit('changemyview').hot(limit=500):
    # Add submission body and title to arg_threads[i]
    arg_threads[submission.id] = []
    arg_threads[submission.id] = arg_threads[submission.id] + nltk.sent_tokenize(submission.selftext)
    
    arg_titles[submission.id] = submission.title + " ID: " + submission.id

    # Remove "replace more" from comments results
    submission.comments.replace_more(limit=None)

    # Get full comment tree under top level comments and add to arg_threads
    for comment in submission.comments.list():
        arg_threads[submission.id] = arg_threads[submission.id] + nltk.sent_tokenize(comment.body)
        
        


In [2]:
import joblib

ngram_model = joblib.load('./models/arg_detection_ngram_model.pkl')
meta_model = joblib.load('./models/arg_detection_meta_model.pkl')


In [ ]:
# Lemmatize all words in a sentence. Uses PoS to identify lemma
def lemmatize_all(sentence):
    wnl = WordNetLemmatizer()
    for word, tag in pos_tag(word_tokenize(sentence)):
        if tag.startswith("NN"):
            yield wnl.lemmatize(word, pos='n')
        elif tag.startswith('VB'):
            yield wnl.lemmatize(word, pos='v')
        elif tag.startswith('JJ'):
            yield wnl.lemmatize(word, pos='a')
        else:
            yield word

# Extract features from a dataset
def extract_features(data_x):
    
    # Predictions based on ngrams from classifier trained on all data
    ngram_predictions = [1 if prediction == "arg" else 0 for prediction in clf_svm.predict(data_x)]
    
    # Sentiment of sentence
    sid = SentimentIntensityAnalyzer()
    sentiment = [sid.polarity_scores(sample)['compound'] for sample in data_x]
    
    # Sentence Length
    sentence_length = [len(word_tokenize(sample)) for sample in data_x]
    
    # Number of each parts of speech
    pos_counts = []
    for sample in data_x:
        nn = 0
        vb = 0
        jj = 0
        for word, pos in pos_tag(word_tokenize(sample)):
            if pos.startswith('NN'):
                nn += 1
            elif pos.startswith('VB'):
                vb += 1
            elif pos.startswith('JJ'):
                jj += 1
        pos_counts.append([nn, vb, jj])
    
    # Combine features into numpy matrix
    return np.column_stack([ngram_predictions, sentiment, sentence_length, pos_counts])

In [ ]:

def find_args(text):
    
    return args

In [7]:



import string
sentence = "sadsad&**"

sentence.translate(str.maketrans('', '', string.punctuation))


'sadsad'

In [111]:

from importlib import reload

from argument_detection import ArgumentPredictor

reload(argument_detection)

argument_predictor = ArgumentPredictor()
argument_predictor.predict_argument("I would tell you that it is more of a follow on to the actions of Obama.")

[nltk_data] Downloading package wordnet to /home/effsy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


AttributeError: 'NoneType' object has no attribute 'lower'

In [ ]:
meta_model.predict()

In [74]:
import json

dataset = []

with open('./labelled_data/1000_labelled_argument_sentences_3.json') as f:
    for line in f:
        json_line = json.loads(line)
        arg = {"text": json_line["content"], "label": json_line["annotation"]["labels"][0]}

        dataset.append(arg)

dataset

# Remove punctuation

import string

dataset = [{"text": sample["text"].translate(str.maketrans('', '', string.punctuation)), "label":sample["label"]} for sample in dataset]
dataset


data = [sample["text"] for sample in dataset]
data
# argument_predictor.predict_argument('The motivation for the age restriction like a lot of the Constitution might have roots in the political situation in Europe in the 1700s')
[argument_predictor.predict_argument(text) for text in data]

AttributeError: 'NoneType' object has no attribute 'lower'

In [25]:
sentence_lemma = "People having good careers, means more revenue for the government and happier citizens."

1 if ngram_model.predict([sentence_lemma])[0] == "arg" else 0

1

In [30]:
def lemmatize_sentence(sentence):
    wnl = WordNetLemmatizer()
    for word, tag in pos_tag(word_tokenize(sentence)):
        if tag.startswith("NN"):
            yield wnl.lemmatize(word, pos='n')
        elif tag.startswith('VB'):
            yield wnl.lemmatize(word, pos='v')
        elif tag.startswith('JJ'):
            yield wnl.lemmatize(word, pos='a')
        else:
            yield word

lemmatize_sentence("People having good careers, means more revenue for the government and happier citizens.")

<generator object lemmatize_sentence at 0x7f7eb60e1678>

In [70]:
argument_predictor.predict_argument("People having good careers, means more revenue for the government and happier citizens.")

array(['not_arg'], dtype='<U7')

In [75]:
meta_model.predict(features)

NameError: name 'features' is not defined

In [119]:

import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import string

import numpy as np
def lemmatize_sentence(sentence):
    wnl = WordNetLemmatizer()
    lemma_array = []

    for word, tag in pos_tag(word_tokenize(sentence)):
        if tag.startswith("NN"):
            lemma_array.append(wnl.lemmatize(word, pos='n'))
        elif tag.startswith('VB'):
            lemma_array.append(wnl.lemmatize(word, pos='v'))
        elif tag.startswith('JJ'):
            lemma_array.append(wnl.lemmatize(word, pos='a'))
        else:
            lemma_array.append(word)
        
        return ' '.join(lemma_array)

def extract_features(sentence):
    # Predictions based on ngrams from classifier trained on all data       

    # Lemmatize
    sentence_lemma = [lemmatize_sentence(sentence)]

    ngram_prediction = 1 if ngram_model.predict(sentence_lemma)[0] == "arg" else 0

    # Sentiment of sentence
    sid = SentimentIntensityAnalyzer()
    sentiment = sid.polarity_scores(sentence)['compound']

    # Sentence Length
    sentence_length = len(word_tokenize(sentence))

    # Number of each parts of speech
    pos_counts = []
    nn = 0
    vb = 0
    jj = 0
    for word, pos in pos_tag(word_tokenize(sentence)):
        if pos.startswith('NN'):
            nn += 1
        elif pos.startswith('VB'):
            vb += 1
        elif pos.startswith('JJ'):
            jj += 1
    pos_counts.append([nn, vb, jj])

    # Combine features into numpy matrix
    return np.column_stack([ngram_prediction, sentiment, sentence_length, pos_counts])

In [110]:

extract_features("People having good careers, means more revenue for the government and happier citizens.")
data[:10]

# [extract_features(text) for text in data[:209]]

data[208]

''

In [121]:
lemmatize_sentence("")

''